## Analyze whether SNWD varies more from year to year or from place to place.

In [2]:
import pandas as pd
import numpy as np
import urllib
import math

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [5]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBBSSBSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
487
+-----------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|          coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|               res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+-----------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|8406.861268207338| 595.8026736315292|-2091.944322965761|   2072.0|SBBSSBSB| 43.8567|-110.5889|       SNWD|0.11229294319897737|0.10783425804107764|0.052867242901368176|0.627692778221

In [6]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [7]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00486440,1970.0,595.802674
1,USC00480140,1964.0,1317.394100
2,USC00486440,1950.0,1162.883467
3,USC00486428,1968.0,-230.778563
4,USC00486430,1955.0,300.589802


In [8]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(100)

station,US1WYTT0009,USC00480140,USC00480603,USC00480605,USC00480778,USC00480865,USC00482054,USC00482242,USC00482375,USC00482725,...,USC00486440,USC00487260,USC00487760,USC00488209,USC00488858,USC00488870,USC00488875,USC00488880,USC00488884,USC00489305
year,,,,,,,,,,,,,,,,,,,,,
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949.0,NaN,567.691387,NaN,NaN,NaN,308.432618,NaN,NaN,NaN,NaN,...,1088.344718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-88.643921
1950.0,NaN,657.598978,NaN,NaN,NaN,-400.149113,NaN,NaN,NaN,NaN,...,1162.883467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,-1236.883732,NaN,NaN,NaN,-1268.337147,NaN,NaN,NaN,NaN,...,-553.078819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952.0,NaN,502.127021,NaN,NaN,NaN,226.489486,NaN,NaN,NaN,NaN,...,806.435424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953.0,NaN,-568.918023,NaN,NaN,NaN,-546.890780,NaN,NaN,NaN,NaN,...,1030.691351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954.0,NaN,-95.251310,NaN,1016.717277,NaN,388.736124,NaN,NaN,NaN,NaN,...,1114.792920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [9]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  1122.69047274
RMS removing mean-by-station=  1006.7957443
RMS removing mean-by-year   =  702.159177023


In [10]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1122.69047274
0 after removing mean by year    = 702.159177023
0 after removing mean by stations= 601.039192384
1 after removing mean by year    = 595.368220259
1 after removing mean by stations= 594.540118584
2 after removing mean by year    = 594.353384834
2 after removing mean by stations= 594.30037449
3 after removing mean by year    = 594.283441102
3 after removing mean by stations= 594.277692132
4 after removing mean by year    = 594.275669506
4 after removing mean by stations= 594.274940397
